In [1]:
!pip install muse_sc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# loading some basic packages
from google.colab import drive
import os
import sys
#import torch
#os.environ['TORCH'] = torch.__version__
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!pip install PhenoGraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.6 MB/s eta 0:00:00


In [4]:
import muse_sc as muse
import phenograph
from sklearn.decomposition import PCA
import numpy as np
from sklearn.metrics.cluster import adjusted_rand_score
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
np.random.seed(0)

Instructions for updating:
non-resource variables are not supported in the long term


We do batch for loop in this case

In [5]:
import pandas as pd

###### process protein modality

data1 = pd.read_csv("/content/drive/MyDrive/spatial_cluster/processed_codex_tonsil/tonsil_features_metadata_47294.csv")
dataa = data1.iloc[:, 4:-3] # drop non-protein columns
#dataa = dataa.drop('nucl',axis = 1)
dataa

,CD38,CD19,CD31,Vimentin,CD22,Ki67,CD8,CD90,CD123,CD15,...,CD63,CD11b,CD1c,CD40,CD274,CD27,CD104,CD273,FAPalpha,Ecadherin
0,0.082069,0.071767,1.000000,0.430528,0.062418,0.057667,0.592763,1.000000,1.000000,1.000000,...,0.708221,1.000000,0.136856,0.233298,0.734386,0.345594,1.000000,0.542104,0.592739,0.725943
1,0.055950,0.000000,0.059087,0.254524,0.023639,0.030778,0.145004,1.000000,0.031251,0.340530,...,0.165686,0.021397,0.038370,0.000000,0.555710,0.030939,0.718946,0.074509,0.232961,0.284406
2,0.012452,0.000000,0.000000,0.103725,0.032043,0.024229,0.039680,0.765956,0.000000,0.107998,...,0.079212,0.000000,0.020039,0.000000,0.464324,0.124583,0.575420,0.000000,0.166899,0.147922
3,0.066589,0.070488,1.000000,0.500190,0.059342,0.055375,0.550346,1.000000,1.000000,1.000000,...,0.715282,1.000000,0.091710,0.197874,0.563294,0.317039,1.000000,0.455486,0.707857,0.659805
4,0.095977,0.012363,1.000000,0.367788,0.058473,0.044813,0.313876,0.713508,0.783856,0.989300,...,0.408118,0.430423,0.060887,0.055072,0.659960,0.149167,0.525709,0.162204,0.430255,0.377030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47289,0.252856,0.016943,0.384003,0.702660,0.068048,0.394010,0.189047,0.335454,0.446136,0.468755,...,0.090013,0.476222,0.117206,1.000000,0.896240,0.434366,0.150436,0.620546,0.696817,0.415139
47290,0.133305,0.367075,0.166159,0.347400,0.702997,0.032621,0.592207,0.707358,0.255965,0.248131,...,0.237204,0.259551,0.157613,0.411775,0.388922,0.189043,0.138195,0.172979,0.458317,0.251480
47291,0.091201,0.218580,0.059986,0.151050,0.470461,0.072847,0.699412,0.012248,0.053091,0.124759,...,0.035418,0.221237,0.144976,0.322006,0.412513,0.181227,0.061962,0.043291,0.321641,0.170924
47292,0.125911,0.388456,0.439975,0.255282,0.099554,0.133983,0.169948,0.405484,0.156668,0.405368,...,0.155308,0.573614,0.551334,1.000000,0.686992,0.329285,0.175148,0.983589,0.717549,0.348095


In [6]:
####### process imaging modality

datab_full = np.load('/content/drive/MyDrive/MaxFuse_related/MUSE_related/Inception_img_feature_tar_full_tonsilv2_47294.npy')
## change img feature sequence to real
llreal = np.load('/content/drive/MyDrive/MaxFuse_related/MUSE_related/realidx_full_tonsilv2.npy')
datab_full = datab_full[llreal,:]

In [7]:
###### this is pca on feature mod and image mod

latent_dim = 20
view_a_feature = PCA(n_components=latent_dim).fit_transform(dataa)
### this is pca on img mod
latent_dim = 100
view_b_feature = PCA(n_components=latent_dim).fit_transform(datab_full)
### then default clustering
view_a_label, _, _ = phenograph.cluster(view_a_feature)
view_b_label, _, _ = phenograph.cluster(view_b_feature)


Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 12.538349866867065 seconds
Jaccard graph constructed in 17.948214292526245 seconds
Wrote graph to binary file in 0.8376038074493408 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.822761
After 4 runs, maximum modularity is Q = 0.824128
After 8 runs, maximum modularity is Q = 0.825221
After 10 runs, maximum modularity is Q = 0.826339
Louvain completed 30 runs in 71.99264335632324 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 103.58009147644043 seconds
Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 23.798267364501953 seconds
Jaccard graph constructed in 16.890637636184692 seconds
Wrote graph to binary file in 0.7972526550292969 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.752463
After 2 runs, maximum modularity is Q = 0.753633
Afte

In [8]:
################################# loop batch version, random sample
for i in range(5):

  # random sub
  indices = np.random.choice(datab_full.shape[0], 10000, replace=False)
  dataa_sub = dataa.iloc[indices,:]
  datab_sub = datab_full[indices,:]
  view_a_label_sub = view_a_label[indices]
  view_b_label_sub = view_b_label[indices]

  muse_feature, reconstruct_x, reconstruct_y, \
  latent_x, latent_y = muse.muse_fit_predict(dataa_sub.to_numpy(),
                                            datab_sub,
                                            view_a_label_sub,
                                            view_b_label_sub,
                                            latent_dim=30,
                                            n_epochs=500,
                                            lambda_regul=5,
                                            lambda_super=5)
  name = 'muse_final_tonsil_rdm10k_' + str(i) + '.npy'
  np.save('/content/drive/MyDrive/MaxFuse_related/MUSE_related/' + name, muse_feature)

++++++++++ MUSE for multi-modality single-cell analysis ++++++++++
MUSE initialization
epoch: 0, 	 total loss: 14097.79980,	 reconstruction loss: 13807.05762,	 sparse penalty: 58.14851
epoch: 50, 	 total loss: 3227.66772,	 reconstruction loss: 3074.90820,	 sparse penalty: 30.55193
epoch: 100, 	 total loss: 2574.57935,	 reconstruction loss: 2487.31177,	 sparse penalty: 17.45353
epoch: 150, 	 total loss: 2356.88574,	 reconstruction loss: 2300.65112,	 sparse penalty: 11.24694
epoch: 0, 	 total loss: 2324.57178,	 reconstruction loss: 2211.87427,	 sparse penalty: 7.98482,	 x triplet: 8.27704,	 y triplet: 6.27764
epoch: 50, 	 total loss: 2237.77856,	 reconstruction loss: 2147.80591,	 sparse penalty: 6.97151,	 x triplet: 5.97663,	 y triplet: 5.04645
epoch: 100, 	 total loss: 2183.65649,	 reconstruction loss: 2101.01587,	 sparse penalty: 6.09746,	 x triplet: 5.63088,	 y triplet: 4.79974
epoch: 150, 	 total loss: 2141.83130,	 reconstruction loss: 2064.15332,	 sparse penalty: 5.42283,	 x triplet